In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

memory = ConversationBufferMemory(
    return_messages=True,
)

cache_dir = LocalFileStore("./.cache/")

llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator = "\n",
    chunk_size=600,
    chunk_overlap=100,
)

# loader = UnstructuredFileLoader("./file/test.txt")
loader = UnstructuredFileLoader("./file/chapter3.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

# chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="refine",
#     # chain_type="stuff",
#     retriever=vectorstore.as_retriever()
# )

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)


chain = {"context":retriever, "question":RunnablePassthrough()} | prompt | llm

chain.invoke("Aaronson 은 유죄인가요?")

AIMessage(content='제가 알기로는 Aaronson이 유죄인지 아닌지에 대한 정보는 제가 가지고 있지 않습니다.')

In [37]:
chain.invoke("그가 테이블에 어떤 메시지를 썼나요?")

AIMessage(content='그가 쓴 메시지는 "FREEDOM IS SLAVERY"와 "TWO AND TWO MAKE FIVE"입니다.')

In [38]:
chain.invoke("Julia 는 누구인가요?")

AIMessage(content='제가 알기로는 Julia는 윈스턴과 함께 이야기하는 여성 캐릭터입니다.')